In [1]:
import pandas as pd

#### IMPORT THE DATA AND REMOVE NAMELESS BOROUGHS

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0]
df = df[df['Borough']!='Not assigned']
df = df.reset_index(drop=True)

#### GROUP NEIGHBOURS FROM SAME BOROUGH

In [3]:
grouped = df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(lambda x: "%s"%', '.join(x))
df1 = pd.DataFrame(grouped)
df1 = df1.reset_index()

In [4]:
def fun(x):
    if x['Neighborhood']=='Not assigned':
        return x['Borough']
    else:
        return x['Neighborhood']

#### RENAME 'NOT ASSIGNED' TO BOROUGH IN NEIGHBORHOOD

In [5]:
df1["new_neighborhood"] = df1.apply(lambda x:fun(x),axis=1)
df1 = df1.drop('Neighborhood',axis=1)
df1.rename(columns={'new_neighborhood':'Neighborhood'},inplace=True)

In [6]:
df1.shape

(103, 3)

In [7]:
geo = pd.read_csv('https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')

#### EXTRACT LATITUDE AND LONGITUDE COORDINATES FOR EACH POSTAL CODE

In [8]:
lat = []
long = []
for index,row in df1.iterrows():
    lat.append(geo[geo["Postal Code"]==row["Postal Code"]][["Latitude"]].iloc[0,0])
    long.append(geo[geo["Postal Code"]==row["Postal Code"]][["Longitude"]].iloc[0,0])
coord = {"Latitude":lat,"Longitude":long}
coord_df = pd.DataFrame(coord)
df2 = pd.concat([df1, coord_df], axis=1).reindex(df1.index)

In [9]:
df2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
